In [2]:
import pandas as pd
import numpy as np
from datetime import datetime 

### Function

In [3]:
def read_until_null_excel(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    null_row_index = df.isnull().all(axis=1).idxmax() if df.isnull().all(axis=1).any() else None

    if null_row_index is not None:
        df = df.iloc[:null_row_index]
    return df

def is_group_in_item(item, group):
    item_str = str(item)
    group_str = str(group)
    return group_str in item_str
def map_group_numbers(df1, df2, item_column, vendor_column, groups_column):
    result = []
    for _, row in df1.iterrows():
        item = row[item_column]
        vendor = str(row[vendor_column])
        matched_group = ''
        for _, group_row in df2.iterrows():
            group_vendor = str(group_row[vendor_column])
            if group_vendor == vendor:
                group = group_row[groups_column]
                if is_group_in_item(item, group):
                    matched_group = group
                    print(f"Match found: Item '{item}' (Vendor: '{vendor}') matches Group '{group}'")
                    break
        result.append(matched_group)
        if not matched_group:
            print(f"No match for Item '{item}' (Vendor: '{vendor}')")
    return result

def first_non_null_column_name(df, start_col, end_col, new_col_name):
    columns_to_check = df.loc[:, start_col:end_col].columns

    def find_first_non_null(row):
        for col in columns_to_check:
            if pd.notnull(row[col]) and row[col] != 0:
                return col
        return None

    df[new_col_name] = df.apply(find_first_non_null, axis=1)
    return df

#Sum pairs
def sum_pairs(df):
    # Initialize an empty list to store the results
    result_list = []
    
    # Separate numeric and non-numeric columns
    numeric_cols = df.select_dtypes(include='number').columns
    non_numeric_cols = df.select_dtypes(exclude='number').columns
    
    # Iterate through the DataFrame in steps of 2
    for i in range(0, len(df), 2):
        # Check if the next row exists
        if i + 1 < len(df):
            # Sum the current row and the next row for numeric columns
            summed_row = df.iloc[i:i+2][numeric_cols].sum()
            # Keep the non-numeric column from the first row
            non_numeric = df.iloc[i][non_numeric_cols]
            # Combine numeric and non-numeric results
            combined = pd.concat([summed_row, non_numeric])
            # Append the result to the list
            result_list.append(combined)
    
    # Convert the list of results to a DataFrame
    result = pd.DataFrame(result_list)
    
    return result

def returnS_F(df_1):
    result_list = []
    for row in range(0, len(df_1), 2):
        found = False
        for col in range(4, len(df_1.columns) - 2):
            if df_1.iloc[row, col] > 0:
                result_list.append(df_1.columns[col])
                found = True
                break
        if not found:
            if row + 1 < len(df_1):
                for col in range(4, len(df_1.columns) - 2):
                    if df_1.iloc[row + 1, col] > 0:
                        result_list.append(df_1.columns[col])
                        break
    result = pd.DataFrame(result_list, columns=['New-Outcome'])
    return result

def highlight_condition_Arca_EC(df, s1, s2,date_columns_1):
    # Define a function to apply to each row
    def highlight(row):
        # Initialize a list of empty styles for each column
        styles = [''] * len(row)
        
        # Apply red background if the condition is met
        if row[s1] < row[s2]:
            styles[df.columns.get_loc(s1)] = 'background-color: red'
            styles[df.columns.get_loc(s2)] = 'background-color: red'
        
        return styles
    df[date_columns_1] = df[date_columns_1].astype(str)
    df[date_columns_1] = df[date_columns_1].replace('1999-01-01', '0')
    df['Vendor #'] = df['Vendor #'].astype(int)
    # Apply the function row-wise
    return df.style.apply(highlight, axis=1)

### Import File

In [4]:
df_2 = pd.read_csv('Production ScheduleDoLuong21551.csv', skiprows = 6)
df_2['Vendor #'] = df_2['Vendor #'].astype(float)

In [5]:
file_input = 'April HP 2024 Price Confirmation Roll-Upv 7.28.xlsx'
sheet_name = 'April HP 2024 Price Confirmatio'
df_1 = read_until_null_excel(file_input, sheet_name)

/Users/thaongan/Documents/Python/ToolAutomation/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Modified df_1 (File SmartSheet)

In [6]:
vendor_take = df_1['Vendor #'].unique()
df_2 = df_2[df_2['Vendor #'].isin(vendor_take)]

In [7]:
split_columns = df_1['Group Number'].str.split(' ', expand=True)

In [8]:
if split_columns.shape == 2:
    split_columns.columns = ['Group Number Split','Additional Component']
else:
    split_columns = split_columns.rename(columns={0: 'Group Number Split'})
    split_columns['Additional Component'] = None

In [9]:
df_1 = pd.concat([df_1, split_columns], axis=1)
group_number_list = df_1['Group Number Split'].unique().tolist()
df_1.drop(columns=['Group Number'], inplace=True)
df_1 = df_1.rename(columns={'Group Number Split': 'Group Number'})

### Modified df_2 (File System)

In [10]:
df_2['Item #'] = df_2['Item #'].str.replace('="', '').str.replace('"', '')

In [11]:
df_2_sample = df_2.loc[(df_2['S/F/P'] == 'F') | (df_2['S/F/P'] == 'S')]
df_2_sample = df_2_sample.loc[:,df_2_sample.columns[4]:]

In [12]:
df_2_sample  = df_2_sample.astype(float)

In [13]:
df = pd.DataFrame(df_2_sample)
#result = sum_pairs(df)
result = returnS_F(df)

In [14]:
df_2 = df_2.loc[df_2['S/F/P'] == 'F'].reset_index(drop=True)

In [15]:
# df_2.iloc[:, 4:].astype(float)
# df_2.iloc[:, 4:] = np.nan

In [16]:
repeated_result = result.reindex(result.index.repeat(2)).reset_index(drop=True)

In [17]:
df_2['New_OutCome'] = repeated_result

In [18]:
item_column = 'Item #'
vendor_column = 'Vendor #'
groups_column = 'Group Number'
df_2['Group Number'] = map_group_numbers(df_2, df_1, item_column, vendor_column, groups_column)

No match for Item '01134' (Vendor: '900352.0')
No match for Item '01222' (Vendor: '900352.0')
No match for Item '01235' (Vendor: '900352.0')
No match for Item '01266' (Vendor: '900352.0')
No match for Item '01279' (Vendor: '900352.0')
No match for Item '01281' (Vendor: '900352.0')
No match for Item '01282' (Vendor: '900352.0')
No match for Item '02193' (Vendor: '900352.0')
No match for Item '03034' (Vendor: '900352.0')
No match for Item '03948' (Vendor: '900352.0')
No match for Item '04483' (Vendor: '900352.0')
No match for Item '04484' (Vendor: '900352.0')
No match for Item '09646' (Vendor: '900352.0')
No match for Item '09697' (Vendor: '900352.0')
No match for Item '100-17' (Vendor: '617616.0')
No match for Item '100-18' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '10516565' (Vendor: '900352.0')
No match for Item '10517693' (Vendor: '900352.0')
No

### Create File Check

In [19]:
df_check = df_2.loc[df_2['Group Number'] != '']

In [20]:
df_check.loc[:, 'Arcadia ETD System'] = np.where(df_check['Whse'].isin({'1'}), 1, 0)
df_check.loc[:, 'EC ETD System'] = np.where(df_check['Whse'].isin({'15', '17', 'ECR'}), 1, 0)
df_check.loc[:, 'WC ETD System'] = np.where(df_check['Whse'].isin({'42', '28', '5'}), 1, 0)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/385469515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check.loc[:, 'Arcadia ETD System'] = np.where(df_check['Whse'].isin({'1'}), 1, 0)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/385469515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check.loc[:, 'EC ETD System'] = np.where(df_check['Whse'].isin({'15', '17', 'ECR'}), 1, 0)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/385469515.py:3

In [21]:
df_check_v2  = pd.merge(df_check, df_1[['Vendor #','Group Number','Additional Component','Arcadia ETD','EC ETD', 'WC ETD','Categories']], on =['Group Number','Vendor #'], how = 'left')

In [22]:
df_check_v3 = first_non_null_column_name(df_check_v2, ' 7/20', ' 12/7', 'Date_First_Value')

In [23]:
current_year = datetime.now().strftime('%Y')
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].str.strip()
df_check_v3['Date_First_Value']= df_check_v3['Date_First_Value'] + '/' + current_year

In [24]:
df_check_v3['Date_First_Value'] = pd.to_datetime(df_check_v3['Date_First_Value'], format='%m/%d/%Y', errors='coerce')
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].dt.strftime('%Y-%m-%d')

In [25]:
df_check_v3['Arcadia ETD System Final'] = np.where(df_check_v3['Arcadia ETD System'] == 1, df_check_v3['Date_First_Value'],0)
df_check_v3['EC ETD System Final'] = np.where(df_check_v3['EC ETD System'] == 1, df_check_v3['Date_First_Value'],0)
df_check_v3['WC ETD System Final'] = np.where(df_check_v3['WC ETD System'] == 1, df_check_v3['Date_First_Value'],0)

In [26]:
df_filtered = df_check_v3[['Categories','Item #','Whse','Group Number','Additional Component','Vendor #','Arcadia ETD System','EC ETD System','WC ETD System','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final', 'Arcadia ETD','EC ETD','WC ETD']]

In [27]:
df_filtered['Arcadia ETD'] = df_filtered['Arcadia ETD'].dt.strftime('%Y-%m-%d')
df_filtered['EC ETD'] = df_filtered['EC ETD'].dt.strftime('%Y-%m-%d')
df_filtered['WC ETD'] = df_filtered['WC ETD'].dt.strftime('%Y-%m-%d')

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/2895289623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Arcadia ETD'] = df_filtered['Arcadia ETD'].dt.strftime('%Y-%m-%d')
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/2895289623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['EC ETD'] = df_filtered['EC ETD'].dt.strftime('%Y-%m-%d')
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/2895289623.py:3: SettingWithCopyWarning: 

In [28]:
df_filtered['Arcadia ETD Smartsheet'] = np.where(df_filtered['Arcadia ETD System'] == 1, df_filtered['Arcadia ETD'], 0)
df_filtered['EC ETD Smartsheet'] = np.where(df_filtered['EC ETD System'] == 1, df_filtered['EC ETD'], 0)
df_filtered['WC ETD Smartsheet'] = np.where(df_filtered['WC ETD System'] == 1, df_filtered['WC ETD'], 0)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/170018666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Arcadia ETD Smartsheet'] = np.where(df_filtered['Arcadia ETD System'] == 1, df_filtered['Arcadia ETD'], 0)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/170018666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['EC ETD Smartsheet'] = np.where(df_filtered['EC ETD System'] == 1, df_filtered['EC ETD'], 0)
/var/folders/c7/zh_6w0ts24d8rh3ts

In [29]:
df_filtered_next = df_filtered[['Categories','Item #','Whse','Group Number','Additional Component','Vendor #','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']]

In [30]:
df_filtered_next['Check True/False'] = np.where(
    (df_filtered_next['Arcadia ETD System Final'] == df_filtered_next['Arcadia ETD Smartsheet']) &
    (df_filtered_next['EC ETD System Final'] == df_filtered_next['EC ETD Smartsheet']) &
    (df_filtered_next['WC ETD System Final'] == df_filtered_next['WC ETD Smartsheet']),
    'True', 'False'
)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_30924/1266345110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_next['Check True/False'] = np.where(


In [31]:
df_filtered_next = df_filtered_next.loc[df_filtered_next['Whse'].isin({'1','15','17','ECR','42','28','5'})]

In [32]:
conditions = [
    df_filtered_next['Whse'].isin({'1'}),
    df_filtered_next['Whse'].isin({'15', '17', 'ECR'})
]

# Define choices
choices = [1, 2]

# Use np.select to assign values based on conditions
df_filtered_next['Type'] = np.select(conditions, choices, default=3)

In [33]:
df_filtered_next['Additional Component'] = df_filtered_next['Additional Component'].fillna('None')

In [34]:
# Replace non-date strings with NaN
date_columns = ['Arcadia ETD System Final', 'EC ETD System Final', 'WC ETD System Final']
df_filtered_next[date_columns] = df_filtered_next[date_columns].replace('0', np.nan)
# Fill NaN values with a placeholder date for aggregation purposes
placeholder_date = '9999-12-31'
df_filtered_next[date_columns] = df_filtered_next[date_columns].fillna(placeholder_date)

# Convert date columns to datetime
df_filtered_next[date_columns] = df_filtered_next[date_columns].apply(pd.to_datetime, format='%Y-%m-%d', errors='coerce')
df_new = df_filtered_next.groupby(['Categories','Item #', 'Group Number','Additional Component','Type','Vendor #','Check True/False','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']).agg({
    'Whse': lambda x: set(x),
    'Arcadia ETD System Final': 'min',
    'EC ETD System Final': 'min',
    'WC ETD System Final': 'min'
}).reset_index()
#turn from datetime to object 
df_new[date_columns] = df_new[date_columns].replace(pd.NaT,'0')

In [35]:
df_new = df_new[['Categories','Item #','Whse','Vendor #','Group Number','Additional Component','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet','Check True/False']]

In [36]:
df_new

,Categories,Item #,Whse,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet,Check True/False
0,Accessories,A8010383,{1},622820.0,A8010383,None,2024-08-24 00:00:00,0,0,2024-08-24,0,0,True
1,Accessories,A8010383,"{ECR, 15, 17}",622820.0,A8010383,None,0,2024-08-24 00:00:00,0,0,2024-08-24,0,True
2,Accessories,A8010383,"{28, 5}",622820.0,A8010383,None,0,0,2024-09-07 00:00:00,0,0,2024-09-07,True
3,Accessories,A8010384,{1},622820.0,A8010384,None,2024-08-24 00:00:00,0,0,2024-08-24,0,0,True
4,Accessories,A8010384,"{ECR, 15, 17}",622820.0,A8010384,None,0,2024-08-24 00:00:00,0,0,2024-08-24,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Outdoor,P704-854,"{ECR, 15, 17}",652525.0,P704,None,0,2024-11-02 00:00:00,0,0,2024-11-02,0,True
267,Outdoor,P704-854,"{28, 5}",652525.0,P704,None,0,0,2024-11-16 00:00:00,0,0,2024-11-16,True
268,Outdoor,P704-877,{1},652525.0,P704,None,2024-10-05 00:00:00,0,0,2024-11-02,0,0,False
269,Outdoor,P704-877,"{ECR, 15, 17}",652525.0,P704,None,0,2024-11-02 00:00:00,0,0,2024-11-02,0,True


In [37]:
df_new_1 = df_new

In [38]:
date_columns_1 = ['Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']

In [39]:
df_new_1[date_columns_1] = df_new_1[date_columns_1].astype(str)
df_new_1[date_columns_1] = df_new_1[date_columns_1].replace('0',np.nan)
place_nan = '1999-01-01'
df_new_1[date_columns_1] = df_new_1[date_columns_1].fillna(place_nan)
df_new_1[date_columns_1] = df_new_1[date_columns_1].apply(pd.to_datetime, format='%Y-%m-%d')

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 00:00:00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [40]:
df_new_1 = df_new_1[['Categories','Item #','Vendor #','Group Number','Additional Component','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']]


In [41]:
df_new_1 = df_new_1.groupby(['Categories', 'Item #','Vendor #','Group Number','Additional Component'])[date_columns_1].max().reset_index()

In [153]:
#Nếu Date đi Arca < Date đi EC thì highlight lên giúp t

In [42]:
df_new_1

,Categories,Item #,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet
0,Accessories,A8010383,622820.0,A8010383,None,2024-08-24 00:00:00,2024-08-24 00:00:00,2024-09-07 00:00:00,2024-08-24,2024-08-24,2024-09-07
1,Accessories,A8010384,622820.0,A8010384,None,2024-08-24 00:00:00,2024-08-24 00:00:00,2024-09-07 00:00:00,2024-08-24,2024-08-24,2024-09-07
2,Bedroom,B895-31,653746.0,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
3,Bedroom,B895-36,653746.0,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
4,Bedroom,B895-46,653746.0,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
...,...,...,...,...,...,...,...,...,...,...,...
85,Outdoor,P704-702,652525.0,P704,None,2024-10-05 00:00:00,2024-11-02 00:00:00,2024-11-16 00:00:00,2024-11-02,2024-11-02,2024-11-16
86,Outdoor,P704-821,652525.0,P704,None,2024-10-05 00:00:00,2024-11-02 00:00:00,2024-11-16 00:00:00,2024-11-02,2024-11-02,2024-11-16
87,Outdoor,P704-846,652525.0,P704,None,2024-10-05 00:00:00,2024-11-02 00:00:00,2024-11-16 00:00:00,2024-11-02,2024-11-02,2024-11-16
88,Outdoor,P704-854,652525.0,P704,None,2024-10-05 00:00:00,2024-11-02 00:00:00,2024-11-16 00:00:00,2024-11-02,2024-11-02,2024-11-16


In [43]:
styled_df = highlight_condition_Arca_EC(df_new_1, 'Arcadia ETD System Final', 'EC ETD System Final',date_columns_1)

In [44]:
styled_df

,Categories,Item #,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet
0,Accessories,A8010383,622820,A8010383,None,2024-08-24 00:00:00,2024-08-24 00:00:00,2024-09-07 00:00:00,2024-08-24,2024-08-24,2024-09-07
1,Accessories,A8010384,622820,A8010384,None,2024-08-24 00:00:00,2024-08-24 00:00:00,2024-09-07 00:00:00,2024-08-24,2024-08-24,2024-09-07
2,Bedroom,B895-31,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
3,Bedroom,B895-36,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
4,Bedroom,B895-46,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
5,Bedroom,B895-54,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
6,Bedroom,B895-56,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
7,Bedroom,B895-57,653746,B895,None,0,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
8,Bedroom,B895-58,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
9,Bedroom,B895-93,653746,B895,None,2024-08-17 00:00:00,2024-08-17 00:00:00,2024-08-31 00:00:00,2024-08-10,2024-08-10,2024-08-24
